In [1]:
# pip install pymorphy2 >> None

In [2]:
import pandas as pd
import numpy as np
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# задаем директорию
file_path = '../datasets/'

In [4]:
# открываем файлы с данными
posts = pd.read_csv(os.path.join(file_path, 'posts.csv'), index_col=0)
profiles = pd.read_csv(os.path.join(file_path, 'profiles.csv'), index_col=0)

In [5]:
# открываем датафрейм с постами
posts.head(2)

,user_id,text,likes,comments,reposts
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [6]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [7]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [8]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [9]:
# удалим посты на украинском языке
# определяем шаблон для украинских символов (по специфичным для данного языка символам)
ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# создаем маску, указывающую строки, в которых столбец "post" содержит текст на украинском языке
mask = posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)

# сохраняем в датафрейме только строки, в которых маска имеет значение False
posts = posts[~mask]

In [10]:
# сохраняем хэштэги в отдельный столбец перед их удалением из постов
posts['hashtags'] = posts['post'].str.findall(r'#([^\s]+)').apply(lambda x: ', '.join(x))

In [11]:
%%time
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

CPU times: total: 2min 5s
Wall time: 2min 9s


In [12]:
# удаляем слова, которые идут после хэш-тэга
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'#[^\s]+', ' ', x))

In [13]:
# производим замену дефиса на пробел
posts["post_lemmatized"] = posts["post_lemmatized"].str.replace("-", " ")

In [14]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)
# только русские буквы и пробелы
posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', ' ', regex=True) 

In [15]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
file_path_words = os.path.join(file_path, 'stop_words.txt')
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [17]:
'''если НЕ оставляем в постах английские слова'''
# определяем шаблон регулярного выражения для русских букв
pattern = '[^а-яА-ЯёЁ]'
# создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
# фильтруем датафрейм, используя маску
posts = posts[mask]

In [18]:
posts.head()

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0,"podcast, миражи",говоря подкаст мираж доступный соцсеть вконтакте разобраться прикрутить
2,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0,"вакансия, vacancy, illustration, иллюстрация, project, проект",честной народ искать векторный иллюстратор проект возможный длительный сотрудничество итогу удаленно уровень иллюстрация пример репост ести уметь рисовать написать личка
3,ali-wodan,"Новый пост подкаста ""Миражи"" на всех платформах:В аудио: https://lnkd.in/giWuSVNВ инстаграме: https://lnkd.in/gbV6yxKН #automotive # # #cars #а патреоне: https://lnkd.in/gfShqTU#саморазвитие #психология #психологиячеловека #психологияжизни #психологияличности #личностныйрост #подкаст #подкастмиражи #маркетинг #осознанность #мысли #мысливслух #установки #влияние #подсознаниеИспользованные звуки, музыка и картинки:Аудиоцитата из фильма ""Формула любви"" Марка ЗахароваJardins du Luxembourg by Jahzzar is licensed under a Attribution-ShareAlike 3.0 International License.prisoner by Luis Prado from the Noun Project",1,0,0,"automotive, cars, а, саморазвитие, психология, психологиячеловека, психологияжизни, психологияличности, личностныйрост, подкаст, подкастмиражи, маркетинг, осознанность, мысли, мысливслух, установки, влияние, подсознаниеИспользованные",пост подкаст миражи платформах аудио инстаграме патреоне звуки музыка картинки аудиоцитат фильм формула любви марк захарова
4,ali-wodan,"Подкаст Миражи, Эпизод 13Ошибка невозвратных затратpodcast.ru/1539345144#подкаст #podcast #it #podcasts #psychology #психология #miragespodcast",2,0,0,"подкаст, podcast, it, podcasts, psychology, психология, miragespodcast",подкаст миражи эпизод ошибка невозвратный затрат
5,ali-wodan,Новый эпизод об эффекте ИКЕА на всех платформах podcast.ru/1539345144,1,0,0,,эпизод эффект икеа платформа


In [19]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2275 entries, 0 to 8894
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          2275 non-null   object
 1   post             2275 non-null   object
 2   likes            2275 non-null   object
 3   comments         2275 non-null   int64 
 4   reposts          2275 non-null   int64 
 5   hashtags         2275 non-null   object
 6   post_lemmatized  2275 non-null   object
dtypes: int64(2), object(5)
memory usage: 142.2+ KB


Объединяем датафреймы

In [20]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [21]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [22]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [23]:
# удаляем из столбца likes точки, запятые и пробелы
df["likes"] = df["likes"].replace(r'\.|\,|\s', '', regex=True)

# меняем тип данных сотлбца likes на integer
df["likes"] = df["likes"].astype("int64")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2264 entries, 0 to 2274
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2264 non-null   object
 1   post              2264 non-null   object
 2   likes             2264 non-null   int64 
 3   comments          2264 non-null   int64 
 4   reposts           2264 non-null   int64 
 5   hashtags          2264 non-null   object
 6   post_lemmatized   2264 non-null   object
 7   user_name         2264 non-null   object
 8   user_head         2264 non-null   object
 9   user_work         2176 non-null   object
 10  user_position     2264 non-null   object
 11  user_tags         574 non-null    object
 12  user_location     2257 non-null   object
 13  user_viewers      2264 non-null   object
 14  user_contacts     2264 non-null   object
 15  user_common_info  2075 non-null   object
dtypes: int64(3), object(13)
memory usage: 300.7+ KB


In [25]:
# Сохраняем датафрейм
df.to_csv(os.path.join(file_path, 'linkedin.csv'))

# Сохраняем датафрейм лемматизации
df[[
    'user_id', 'post', 'post_lemmatized', 'likes', 'comments', 'reposts', 'hashtags'
]].to_csv(os.path.join(file_path, 'post_lemmatized.csv'))

In [26]:
df.sample(5)

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
963,yekaterina-raipova-b13b73a0,"Начальник отдела внутренних коммуникаций - заместитель начальника управления г. Астана ТОО ""Корпорация Казахмыс"" Обязанности:• Формирование и поддержание в актуальном состоянии карты стейкхолдеров по каналам и процессам подразделения (коммуникации, этика, бренд и корпоративные события)• Формирование, актуализация стратегии внутренних коммуникаций, бюджета направления и подготовка отчетности по нему• Проведение коучинга для ключевых спикеров относительно стратегии внутренних коммуникаций и бренда работодателя • Оказание методологической поддержки проектам внутри компании (в рамках управления изменениями Необходимые компетенции:• Высшее профессиональное образование (PR, журналистика, HR, менеджмент);• Наличие курсов повышения квалификации (сертификаты не старше 3-х лет);• Опыт выстраивания отношений с топ-менеджерами;• Опыт работы с обратной связью от руководителей;• Опыт взаимодействия с различными группами стейкхолдерами;• Опыт на управленческой должности - не менее 2-х лет;• Опыт работы в промышленных предприятиях и ГМК приветствуется• Знание основ психологии управления персоналом• Аналитическое и стратегическое мышление Знание казахского языка обязательно Что предлагаем мы:• Команда профессионалов цифрового обучения• Сложные и интересные задачи• Конкурентная заработная плата• График работы: 5/2Ekaterina.Raipova@kazakhmys.kz",5,1,0,,начальник отдел внутренний коммуникация заместитель начальник управление тоо корпорация казахмыс обязанности формирование поддержание актуальный состояние карта стейкхолдер канал процесс подразделение коммуникации этика бренд корпоративный события формирование актуализация стратегия внутренний коммуникаций бюджет направление подготовка отчётность проведение коучинга ключевой спикер относительно стратегия внутренний коммуникация бренд работодатель оказание методологический поддержка проект внутри компания рамка управление изменение необходимый компетенции высокий профессиональный образование журналистика менеджмент наличие курс повышение квалификация сертификат старший лет опыт выстраивание отношение топ менеджерами опыт работа связь руководителей опыт взаимодействие различный группа стейкхолдерами опыт управленческий должность лет опыт работа промышленный предприятие гмк приветствуется знание основа психология управление персоналом аналитический стратегический мышление знание казахский язык обязательно предлагать команда профессионал цифровой обучения сложный интересный задачи конкурентный заработный плата график работы,Yekaterina Raipova,Human Resource Management,Kazakhmys Corporation,Менеджер по адаптации персонала,NaN,"Nur-Sultan, Kazakhstan",772,500+,"Достижения: наняла за 4 года болеет 200 -специалистов блока IT (мануальные тестировщики, QA инженеры, Frontц-end разработчики: React, Angular и т.д.; Back-end разработчики: Oracle PL/SQL, Siebel, ЦФТ, .Net/C#, Java, PHP, IBM BPM, Интеграционная Шина Tibco; DevOps инженеры, Администраторы БД Oracle, MS SQL, PostgreSQL и т.д.)."
84,agratoth,"#react #nextjs #typescriptНаписал статью на Хабр, вдруг кому окажется полезной. Тема несложная, асинхронный алертинг в React/NextJS приложении, но вышло вроде неплохо. Любой конструктивной критике буду рад",1,0,0,"react, nextjs, typescriptНаписал",статья хабр оказаться полезной тема несложная асинхронный алертинг приложении выйти вроде неплохо любой конструктивный критика,Anton Berdnikov,CTO/Co-founder,Cloudberry.bi,Chief Technical Officer/Co-founder,NaN,Russia,"4,569",500+,Разработка нейросетей на python/keras.io + tensorflowРазработка микросервисов на golangРазработка высоконагруженных приложенийСистемная разработка на golangСистемная разработка на C/C++
718,enjirouz,"Мечтаешь разработать свою игру и заработать на ней? Тогда посмотри это видео!В нем я поделюсь с тобой, как всего за 7 шагов пройти цикл со